In [ ]:
!pip install pyspark

     |████████████████████████████████| 204.2MB 59kB/s 
     |████████████████████████████████| 204kB 6.7MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612242 sha256=e90324f44225aaf437550cc9bdd993a15b5dd17767c41d0f8174b2a9125093ef
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType
from pyspark.sql import SparkSession
from pyspark import SparkContext
sc = SparkContext()
spark = SparkSession.builder.getOrCreate()



In [ ]:
# StructField("name", dtype, nullable=True)
peopleSchema = StructType([StructField("name", StringType(), True ), StructField("age", IntegerType(), True), StructField("city", StringType(), True)])
people_df = spark.read.format("csv").load(name="voterdata.csv", schema=peopleSchema) # อันนี้ใส่ Schema มั่วๆเอา

# Read csv
sdf = spark.read.csv("voterdata.csv", inferSchema=True, header=True)
sdf = sdf.withColumn("AGENDA_ITEM_NUMBER", sdf["AGENDA_ITEM_NUMBER"].cast("integer"))

# Add column
sdf = sdf.withColumn("ITEM+100", sdf["AGENDA_ITEM_NUMBER"]+100)
display(sdf.show(3))

# Drop column
sdf = sdf.drop(sdf["AGENDA_ITEM_NUMBER"])
display(sdf.show(3))

# Count number rows
display(sdf.count())

+----------+------------------+---------+--------+-------------+-------------------+---------+------------------+-----------------------+------------------+--------------------+--------+
|      DATE|AGENDA_ITEM_NUMBER|ITEM_TYPE|DISTRICT|        TITLE|         VOTER NAME|VOTE CAST|FINAL ACTION TAKEN|AGENDA ITEM DESCRIPTION|         AGENDA_ID|             VOTE_ID|ITEM+100|
+----------+------------------+---------+--------+-------------+-------------------+---------+------------------+-----------------------+------------------+--------------------+--------+
|02/08/2017|                 1|   AGENDA|      13|Councilmember|  Jennifer S. Gates|      N/A|  NO ACTION NEEDED|          Call to Order|020817__Special__1|020817__Special__...|     101|
|02/08/2017|                 1|   AGENDA|      14|Councilmember| Philip T. Kingston|      N/A|  NO ACTION NEEDED|          Call to Order|020817__Special__1|020817__Special__...|     101|
|02/08/2017|                 1|   AGENDA|      15|        Mayor|M

None

+----------+---------+--------+-------------+-------------------+---------+------------------+-----------------------+------------------+--------------------+--------+
|      DATE|ITEM_TYPE|DISTRICT|        TITLE|         VOTER NAME|VOTE CAST|FINAL ACTION TAKEN|AGENDA ITEM DESCRIPTION|         AGENDA_ID|             VOTE_ID|ITEM+100|
+----------+---------+--------+-------------+-------------------+---------+------------------+-----------------------+------------------+--------------------+--------+
|02/08/2017|   AGENDA|      13|Councilmember|  Jennifer S. Gates|      N/A|  NO ACTION NEEDED|          Call to Order|020817__Special__1|020817__Special__...|     101|
|02/08/2017|   AGENDA|      14|Councilmember| Philip T. Kingston|      N/A|  NO ACTION NEEDED|          Call to Order|020817__Special__1|020817__Special__...|     101|
|02/08/2017|   AGENDA|      15|        Mayor|Michael S. Rawlings|      N/A|  NO ACTION NEEDED|          Call to Order|020817__Special__1|020817__Special__...|  

None

44625

In [ ]:
# Load csv file from .gz (Winrar) file
aa_dfw_df = spark.read.format('csv').options(Header=True).load('AA_DFW_2017_Departures_Short.csv.gz')
display(aa_dfw_df.show(4))

# ไม่เห็นจำเป็นต้องเขียนยาวๆ โง่ๆ
display(spark.read.csv("AA_DFW_2017_Departures_Short.csv.gz", header=True).show(4))
sdf = spark.read.csv("voterdata.csv", inferSchema=True, header=True)


# import string function
import pyspark.sql.functions as F
aa_dfw_df = aa_dfw_df.withColumn('airport', F.lower(aa_dfw_df['Destination Airport']))

# Drop the Destination Airport column
aa_dfw_df = aa_dfw_df.drop(aa_dfw_df['Destination Airport'])

# Show the DataFrame
display(aa_dfw_df.show(5))

# Add tables to catalog
aa_dfw_df.createOrReplaceTempView("flights")
sdf.createOrReplaceTempView("voters")
display(spark.catalog.listTables())

# Can be used with sql query
tested_df = spark.sql("SELECT * FROM flights WHERE airport = 'hnl'")
tested_df.show(4)

+-----------------+-------------+-------------------+-----------------------------+
|Date (MM/DD/YYYY)|Flight Number|Destination Airport|Actual elapsed time (Minutes)|
+-----------------+-------------+-------------------+-----------------------------+
|       01/01/2017|         0005|                HNL|                          537|
|       01/01/2017|         0007|                OGG|                          498|
|       01/01/2017|         0037|                SFO|                          241|
|       01/01/2017|         0043|                DTW|                          134|
+-----------------+-------------+-------------------+-----------------------------+
only showing top 4 rows



None

+-----------------+-------------+-------------------+-----------------------------+
|Date (MM/DD/YYYY)|Flight Number|Destination Airport|Actual elapsed time (Minutes)|
+-----------------+-------------+-------------------+-----------------------------+
|       01/01/2017|         0005|                HNL|                          537|
|       01/01/2017|         0007|                OGG|                          498|
|       01/01/2017|         0037|                SFO|                          241|
|       01/01/2017|         0043|                DTW|                          134|
+-----------------+-------------+-------------------+-----------------------------+
only showing top 4 rows



None

+-----------------+-------------+-----------------------------+-------+
|Date (MM/DD/YYYY)|Flight Number|Actual elapsed time (Minutes)|airport|
+-----------------+-------------+-----------------------------+-------+
|       01/01/2017|         0005|                          537|    hnl|
|       01/01/2017|         0007|                          498|    ogg|
|       01/01/2017|         0037|                          241|    sfo|
|       01/01/2017|         0043|                          134|    dtw|
|       01/01/2017|         0051|                           88|    stl|
+-----------------+-------------+-----------------------------+-------+
only showing top 5 rows



None

[Table(name='flights', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='voters', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

+-----------------+-------------+-----------------------------+-------+
|Date (MM/DD/YYYY)|Flight Number|Actual elapsed time (Minutes)|airport|
+-----------------+-------------+-----------------------------+-------+
|       01/01/2017|         0005|                          537|    hnl|
|       01/01/2017|         0123|                          529|    hnl|
|       01/02/2017|         0005|                          537|    hnl|
|       01/02/2017|         0123|                          532|    hnl|
+-----------------+-------------+-----------------------------+-------+
only showing top 4 rows



In [ ]:
xspark.read.csv("AA_DFW_2017_Departures_Short.csv.gz", header=True).show(4)

+-----------------+-------------+-------------------+-----------------------------+
|Date (MM/DD/YYYY)|Flight Number|Destination Airport|Actual elapsed time (Minutes)|
+-----------------+-------------+-------------------+-----------------------------+
|       01/01/2017|         0005|                HNL|                          537|
|       01/01/2017|         0007|                OGG|                          498|
|       01/01/2017|         0037|                SFO|                          241|
|       01/01/2017|         0043|                DTW|                          134|
+-----------------+-------------+-------------------+-----------------------------+
only showing top 4 rows



In [ ]:
### Working with Parquet 

# Read Parquet 
df = spark.read.format("parquet").load("filename.parquet")
df = spark.read.parquet("filename.parquet")

# Write Parquet
df.write.format("parquet").save("filename.parquet")
df.write.parquet("filename.parquet")

# Add tables to catalog to be used for sql query
df.createOrReplaceTempView("Table_name")


In [ ]:
voter_df = spark.read.csv("voterdata.csv", header=True, inferSchema=True)
voter_df = voter_df.withColumnRenamed("VOTER NAME", "name")

# # เลือกเฉพาะแถวที่ชื่อขึ้นต้นด้วย m
# voter_df.filter(voter_df["name"].like("M%")).show(5)
# voter_df.select("ITEM_TYPE", "name").show(4)
# voter_df.filter(voter_df["DATE"] > '1/1/2017').show(5)
voter_df.where(voter_df["DATE"] > '1/1/2017').show(5)

+--------------------+------------------+-------------+--------+-------------+--------------+---------+------------------+-----------------------+---------+------------+
|                DATE|AGENDA_ITEM_NUMBER|    ITEM_TYPE|DISTRICT|        TITLE|          name|VOTE CAST|FINAL ACTION TAKEN|AGENDA ITEM DESCRIPTION|AGENDA_ID|     VOTE_ID|
+--------------------+------------------+-------------+--------+-------------+--------------+---------+------------------+-----------------------+---------+------------+
|          10/03/2018|                 1|       AGENDA|      12|Councilmember|Sandy  Greyson|      YES|          APPROVED|   Approval of Minut...|100318__1|100318__1_12|
|[APPOINTMENT OF J...|        061318__44| 061318__44_1|    null|         null|          null|     null|              null|                   null|     null|        null|
|[APPOINTMENT OF J...|        061318__44|061318__44_10|    null|         null|          null|     null|              null|                   null|    

In [ ]:
import pyspark.sql.functions as F

voter_df = spark.read.csv("voterdata.csv", header=True, inferSchema=True)
voter_df = voter_df.withColumnRenamed("VOTER NAME", "name")

# Show the distinct VOTER_NAME entries
voter_df.select("name").distinct().show(40, truncate=False)

# Filter voter_df where the VOTER_NAME is 1-20 characters in length
voter_df = voter_df.filter('length(name) > 0 and length(name) < 20')

# Filter out voter_df where the VOTER_NAME contains an underscore
voter_df = voter_df.filter(~ F.col('name').contains('_'))

# Show the distinct VOTER_NAME entries again
voter_df.select("name").distinct().show(40, truncate=False)

# Add a new column called splits separated on whitespace
voter_df = voter_df.withColumn("splits", F.split(voter_df["name"], '\s+'))

# Create a new column called first_name based on the first item in splits
voter_df = voter_df.withColumn("first_name", voter_df["splits"].getItem(0))

# Get the last entry of the splits list and create a column called last_name
voter_df = voter_df.withColumn("last_name", voter_df.splits.getItem(F.size('splits') - 1)) #อย่าลืม มันเริ่มที่ 0 ถ้าขนาด array=3 ตัวสุดท้าย index=2

# Drop the splits column
voter_df = voter_df.drop('splits')

# Show the voter_df DataFrame
voter_df.show()

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|name                                                                                                                                                                                                                                                                                                                                                                                                                       |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# pyspark.sql.functions.when example
import pyspark.sql.functions as F
from pyspark.sql.functions import when

voter_df = spark.read.csv("voterdata.csv", header=True, inferSchema=True)
voter_df = voter_df.withColumnRenamed("VOTER NAME", "name")
voter_df_ex = voter_df.select("DATE", "TITLE", "name")
voter_df_ex = voter_df_ex.withColumn('random_val', when(voter_df["TITLE"]=="Councilmember", F.rand()))
voter_df_ex.show()

# Add a column to voter_df for a voter based on their position (when สามารถ chain ต่อเนื่องได้)
voter_df_ex = voter_df_ex.withColumn('random_val', when(voter_df["TITLE"] == 'Councilmember', F.rand()).when(voter_df["TITLE"] == 'Mayor', 2).otherwise(0))

# Show some of the DataFrame rows
voter_df_ex.show()

# Use the .filter() clause with random_val
voter_df_ex.filter("random_val=0").show() # หรือจะใช้ boolean array ก็ได้


+----------+-------------+-------------------+-------------------+
|      DATE|        TITLE|               name|         random_val|
+----------+-------------+-------------------+-------------------+
|02/08/2017|Councilmember|  Jennifer S. Gates| 0.6113700785651836|
|02/08/2017|Councilmember| Philip T. Kingston| 0.8102519839470655|
|02/08/2017|        Mayor|Michael S. Rawlings|               null|
|02/08/2017|Councilmember|       Adam Medrano|0.19043774220413479|
|02/08/2017|Councilmember|   Casey Thomas, II| 0.3345737334476334|
|02/08/2017|Councilmember|Carolyn King Arnold| 0.6891886102126733|
|02/08/2017|Councilmember|       Scott Griggs| 0.6661727060814254|
|02/08/2017|Councilmember|   B. Adam  McGough|0.30678535440505117|
|02/08/2017|Councilmember|       Lee Kleinman|0.16907165684673686|
|02/08/2017|Councilmember|      Sandy Greyson|0.24403344746448197|
|02/08/2017|Councilmember|  Jennifer S. Gates|0.08406954129531097|
|02/08/2017|Councilmember| Philip T. Kingston| 0.5820154095892

In [ ]:
# User-defined function
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
from pyspark.sql.functions import udf
from random import choice

def reverseString(s):
    return s[::-1]

def sortingCap():
    return choice(["A", "B", "C", "D"])

# เพราะสาเหตุใดไม่ทราบ แต่ reverse ชื่อที่ยังไม่ clean ไม่ได้
voter_df = spark.read.csv("voterdata.csv", header=True, inferSchema=True)
voter_df = voter_df.withColumnRenamed("VOTER NAME", "name")
voter_df = voter_df.filter('length(name) > 0 and length(name) < 20')
voter_df = voter_df.filter(~ F.col('name').contains('_'))
voter_df = voter_df.withColumn("splits", F.split(voter_df["name"], '\s+'))
voter_df = voter_df.withColumn("first_name", voter_df["splits"].getItem(0))
user_df = voter_df.select("DATE", "TITLE", "name", "first_name")

# udf ต้องมี 2 positional arguments: ฟังก์ชัน และชนิดของค่าที่ return
udfReverseString = udf(reverseString, StringType())
udfReverseString2 = udf(lambda s: s[::-1], StringType())
user_df = user_df.withColumn('ReverseName', udfReverseString2(user_df["name"]))
user_df.show()

udfSortingCap = udf(sortingCap, StringType())
user_df.withColumn("Grade", udfSortingCap()).show()

+----------+-------------+-------------------+----------+-------------------+
|      DATE|        TITLE|               name|first_name|        ReverseName|
+----------+-------------+-------------------+----------+-------------------+
|02/08/2017|Councilmember|  Jennifer S. Gates|  Jennifer|  setaG .S refinneJ|
|02/08/2017|Councilmember| Philip T. Kingston|    Philip| notsgniK .T pilihP|
|02/08/2017|        Mayor|Michael S. Rawlings|   Michael|sgnilwaR .S leahciM|
|02/08/2017|Councilmember|       Adam Medrano|      Adam|       onardeM madA|
|02/08/2017|Councilmember|   Casey Thomas, II|     Casey|   II ,samohT yesaC|
|02/08/2017|Councilmember|Carolyn King Arnold|   Carolyn|dlonrA gniK nyloraC|
|02/08/2017|Councilmember|       Scott Griggs|     Scott|       sggirG ttocS|
|02/08/2017|Councilmember|   B. Adam  McGough|        B.|   hguoGcM  madA .B|
|02/08/2017|Councilmember|       Lee Kleinman|       Lee|       namnielK eeL|
|02/08/2017|Councilmember|      Sandy Greyson|     Sandy|      n

In [ ]:
### More Examples and notice how number of partitions affect pyspark.sql.functions.monotonically_increasing_id()
# Select all the unique council voters
df = spark.read.csv("voterdata.csv", header=True, inferSchema=True)
voter_df = df.select(df["VOTER NAME"]).distinct()
voter_df_single = voter_df.repartition(1)

# Count the rows in voter_df
print("\nThere are %d rows in the voter_df DataFrame.\n" % voter_df.count())

# Add a ROW_ID ใช้ฟังก์ชัน F.monotonically_increasing_id 
voter_df = voter_df.withColumn('ROW_ID', F.monotonically_increasing_id())

# Show the rows with 10 highest IDs in the set
voter_df.orderBy(voter_df.ROW_ID.desc()).show(10)

# Print the number of partitions in each DataFrame
print("\nThere are %d partitions in the voter_df DataFrame.\n" % voter_df.rdd.getNumPartitions())
print("\nThere are %d partitions in the voter_df_single DataFrame.\n" % voter_df_single.rdd.getNumPartitions())

# Add a ROW_ID field to each DataFrame
voter_df = voter_df.withColumn('ROW_ID', F.monotonically_increasing_id())
voter_df_single = voter_df_single.withColumn('ROW_ID', F.monotonically_increasing_id())

# Show the top 10 IDs in each DataFrame 
voter_df.orderBy(voter_df.ROW_ID.desc()).show(10)
voter_df_single.orderBy(voter_df_single.ROW_ID.desc()).show(10)


There are 36 rows in the voter_df DataFrame.

+--------------------+-------------+
|          VOTER NAME|       ROW_ID|
+--------------------+-------------+
|        Lee Kleinman|1709396983808|
|  the  final  201...|1700807049217|
|         Erik Wilson|1700807049216|
|  the  final   20...|1683627180032|
| Carolyn King Arnold|1632087572480|
| Rickey D.  Callahan|1597727834112|
|   the   final  2...|1443109011456|
|    Monica R. Alonzo|1382979469312|
|     Lee M. Kleinman|1228360646656|
|   Jennifer S. Gates|1194000908288|
+--------------------+-------------+
only showing top 10 rows


There are 200 partitions in the voter_df DataFrame.


There are 1 partitions in the voter_df_single DataFrame.

+--------------------+-------------+
|          VOTER NAME|       ROW_ID|
+--------------------+-------------+
|        Lee Kleinman|1709396983808|
|  the  final  201...|1700807049217|
|         Erik Wilson|1700807049216|
|  the  final   20...|1683627180032|
| Carolyn King Arnold|1632087572480|


In [ ]:
### Caching
import pyspark.sql.functions as F
voter_df = spark.read.csv("voterdata.csv", header=True, inferSchema=True)
print(voter_df.cache().count())

voter_df = spark.read.csv("voterdata.csv", header=True, inferSchema=True).withColumn("ID", F.monotonically_increasing_id())
voter_df = voter_df.cache()
voter_df.show()

# Check cache
print(voter_df.is_cached)

# Remove cache
voter_df.unpersist()
print(voter_df.is_cached)

44625
+----------+------------------+-----------+--------+-------------+-------------------+---------+------------------+-----------------------+------------------+--------------------+---+
|      DATE|AGENDA_ITEM_NUMBER|  ITEM_TYPE|DISTRICT|        TITLE|         VOTER NAME|VOTE CAST|FINAL ACTION TAKEN|AGENDA ITEM DESCRIPTION|         AGENDA_ID|             VOTE_ID| ID|
+----------+------------------+-----------+--------+-------------+-------------------+---------+------------------+-----------------------+------------------+--------------------+---+
|02/08/2017|                 1|     AGENDA|      13|Councilmember|  Jennifer S. Gates|      N/A|  NO ACTION NEEDED|          Call to Order|020817__Special__1|020817__Special__...|  0|
|02/08/2017|                 1|     AGENDA|      14|Councilmember| Philip T. Kingston|      N/A|  NO ACTION NEEDED|          Call to Order|020817__Special__1|020817__Special__...|  1|
|02/08/2017|                 1|     AGENDA|      15|        Mayor|Michael 

In [ ]:
### How to split objects

# Use OS utilities
#### split -l 10000 -d largefilename chunk- ####
# -l 10000 คือจำนวนบรรทัด
# -d ใช้ตัวเลขเป็น suffix
# largefilename คือชื่อไฟล์
# chunk- เป็น prefix

# เขียนลง parquet
df_csv =  spark.read.csv("voterdata.csv", header=True, inferSchema=True)

# สมมติมี voterdata000.csv - voterdata999.csv
# ใช้ df_csv =  spark.read.csv("voterdata*.csv", header=True, inferSchema=True)

df_csv =  df.withColumnRenamed("VOTER NAME", "name") # ไฟล์ parquet ไม่สามารถใช้ชื่อคอลัมน์มี space ได้ contains invalid character(s) among " ,;{}()\n\t=
df_csv =  df_csv.select("DATE", "name")
df_csv.write.parquet("data3.parquet")
df_pq = spark.read.parquet("data3.parquet")
df_pq.show()

+----------+-------------------+
|      DATE|               name|
+----------+-------------------+
|02/08/2017|  Jennifer S. Gates|
|02/08/2017| Philip T. Kingston|
|02/08/2017|Michael S. Rawlings|
|02/08/2017|       Adam Medrano|
|02/08/2017|   Casey Thomas, II|
|02/08/2017|Carolyn King Arnold|
|02/08/2017|       Scott Griggs|
|02/08/2017|   B. Adam  McGough|
|02/08/2017|       Lee Kleinman|
|02/08/2017|      Sandy Greyson|
|02/08/2017|  Jennifer S. Gates|
|02/08/2017| Philip T. Kingston|
|02/08/2017|Michael S. Rawlings|
|02/08/2017|       Adam Medrano|
|02/08/2017|   Casey Thomas, II|
|02/08/2017|Carolyn King Arnold|
|02/08/2017| Rickey D. Callahan|
|01/11/2017|  Jennifer S. Gates|
|04/25/2018|     Sandy  Greyson|
|04/25/2018| Jennifer S.  Gates|
+----------+-------------------+
only showing top 20 rows



In [ ]:
### Reading Spark configurations

# Name of the Spark application instance
app_name = spark.conf.get("spark.app.name")

# Driver TCP port
driver_tcp_port = spark.conf.get("spark.driver.port")

# Number of join partitions
num_partitions = spark.conf.get('spark.sql.shuffle.partitions')

# Show the results
print("Name: %s" % app_name)
print("Driver TCP port: %s" % driver_tcp_port)
print("Number of partitions: %s" % num_partitions, "\n\n")

# Set Spark config

# Store the number of partitions in variable
departures_df = spark.read.csv('AA_DFW_2017_Departures_Short.csv.gz').distinct()
before = departures_df.rdd.getNumPartitions()

# Configure Spark to use 500 partitions
spark.conf.set('spark.sql.shuffle.partitions', 500)

# Recreate the DataFrame using the departures data file
departures_df = spark.read.csv('AA_DFW_2017_Departures_Short.csv.gz').distinct()

# Print the number of partitions for each instance
print("Partition count before change: %d" % before)
print("Partition count after change: %d" % departures_df.rdd.getNumPartitions())

Name: pyspark-shell
Driver TCP port: 35443
Number of partitions: 200 


Partition count before change: 200
Partition count after change: 500


In [ ]:
### Data Pipelines

# Import the data to a DataFrame
departures_df = spark.read.csv("AA_DFW_2015_Departures_Short.csv.gz", header=True)

# Remove any duration of 0
departures_df = departures_df.filter(departures_df["Actual elapsed time (Minutes)"] > 0)

# Add an ID column
departures_df = departures_df.withColumn('id', F.monotonically_increasing_id())

# Write the file out to JSON format
departures_df.write.json("output.json", mode='overwrite')


In [ ]:
### Removng commented lines

import pyspark.sql.functions as F
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType


# Import the file to a DataFrame and perform a row count
annotations_df = spark.read.csv('annotations.csv.gz', sep="|")
full_count = annotations_df.count()

# Count the number of rows beginning with '#'
comment_count = annotations_df.where(col('_c0').startswith('#')).count()

# Import the file to a new DataFrame, without commented rows
no_comments_df = spark.read.csv('annotations.csv.gz', sep="|", comment='#')

# Count the new DataFrame and verify the difference is as expected
no_comments_count = no_comments_df.count()
print("Full count: %d\nComment count: %d\nRemaining count: %d" % (full_count, comment_count, no_comments_count))

### Removing invalid rows ###

# Split _c0 on the tab character and store the list in a variable
tmp_fields = F.split(annotations_df['_c0'], "\t")

# Create the colcount column on the DataFrame
annotations_df = annotations_df.withColumn('colcount', F.size(tmp_fields))

# Remove any rows containing fewer than 5 fields
annotations_df_filtered = annotations_df.filter(~ (annotations_df.colcount >= 5))

initial_count = 31378
# Count the number of rows
final_count = annotations_df_filtered.count()
print("Initial count: %d\nFinal count: %d" % (initial_count, final_count))

### Splitting into columns ###

# Split the content of _c0 on the tab character (aka, '\t')
split_cols = F.split(annotations_df['_c0'], '\t')

# Add the columns folder, filename, width, and height
split_df = annotations_df.withColumn('folder', split_cols.getItem(0))
split_df = split_df.withColumn('filename', split_cols.getItem(1))
split_df = split_df.withColumn('width', split_cols.getItem(2))
split_df = split_df.withColumn('height', split_cols.getItem(3))

# Add split_cols as a column
split_df = split_df.withColumn('split_cols', split_cols)

### Using F.udf

def retriever(cols, colcount):
  # Return a list of dog data
  return cols[4:colcount]

# Define the method as a UDF
udfRetriever = F.udf(retriever, ArrayType(StringType()))

# Create a new column using your UDF
split_df = split_df.withColumn('dog_list', udfRetriever(split_df["split_cols"], split_df["colcount"]))

# Remove the original column, split_cols, and the colcount
split_df = split_df.drop('_c0').drop('colcount').drop('split_cols')

Full count: 32794
Comment count: 1416
Remaining count: 31378
Initial count: 31378
Final count: 12214


In [ ]:
### Validate rows via join ###

valid_folders_df = spark.read.csv("valid_folders.txt")

# Rename the column in valid_folders_df
valid_folders_df = valid_folders_df.withColumnRenamed("_c0", "folder")

# Count the number of rows in split_df
split_count = split_df.count()

# Join the DataFrames
joined_df = split_df.join(valid_folders_df, "folder")

# Compare the number of rows remaining
joined_count = joined_df.count()
print("Before: %d\nAfter: %d" % (split_count, joined_count))

# Determine the row counts for each DataFrame
split_count = split_df.count()
joined_count = joined_df.count()

# Create a DataFrame containing the invalid rows
invalid_df = split_df.join(joined_df, on='folder', how='leftOuter')

# Validate the count of the new DataFrame is as expected
invalid_count = invalid_df.count()
print(" split_df:\t%d\n joined_df:\t%d\n invalid_df: \t%d" % (split_count, joined_count, invalid_count))

# Determine the number of distinct folder rows removed
invalid_folder_count = invalid_df.select('folder').distinct().count()
print("%d distinct invalid folders found" % invalid_folder_count)



Before: 32794
After: 20945
 split_df:	32794
 joined_df:	20945
 invalid_df: 	3509094
12328 distinct invalid folders found


In [ ]:
### Dog Parsing
# Select the dog details and show 10 untruncated rows
print(joined_df.select("dog_list").show(10, truncate=False))

# Define a schema type for the details in the dog list
DogType = StructType([
	StructField("breed", StringType(), False),
    StructField("start_x", IntegerType(), False),
    StructField("start_y", IntegerType(), False),
    StructField("end_x", IntegerType(), False),
    StructField("end_y", IntegerType(), False)])

# Create a function to return the number and type of dogs as a tuple
def dogParse(doglist):
  dogs = []
  for dog in doglist:
    (breed, start_x, start_y, end_x, end_y) = dog.split(',')
    dogs.append((breed, int(start_x), int(start_y), int(end_x), int(end_y)))
  return dogs

# Create a UDF
udfDogParse = F.udf(dogParse, ArrayType(DogType))

# Use the UDF to list of dogs and drop the old column
joined_df = joined_df.withColumn('dogs', udfDogParse('dog_list')).drop('dog_list')

# Show the number of dogs in the first 10 rows
joined_df.select(F.size('dogs')).show(10)

# Define a UDF to determine the number of pixels per image
def dogPixelCount(doglist):
  totalpixels = 0
  for dog in doglist:
    totalpixels += (dog[3] - dog[1]) * (dog[4] - dog[2])
  return totalpixels

# Define a UDF for the pixel count
udfDogPixelCount = F.udf(dogPixelCount, IntegerType())
joined_df = joined_df.withColumn("dog_pixels", udfDogPixelCount("dogs"))

# Create a column representing the percentage of pixels
joined_df = joined_df.withColumn('dog_percent', (joined_df.dog_pixels / (joined_df.width*joined_df.height)) * 100)

# Show the first 10 annotations with more than 60% dog
joined_df.filter("dog_percent > 60").show(10)

+----------------------------------+
|dog_list                          |
+----------------------------------+
|[affenpinscher,0,9,173,298]       |
|[Border_terrier,73,127,341,335]   |
|[kuvasz,0,0,499,327]              |
|[Great_Pyrenees,124,225,403,374]  |
|[schipperke,146,29,416,309]       |
|[groenendael,168,0,469,374]       |
|[Bedlington_terrier,10,12,462,332]|
|[Lhasa,39,1,499,373]              |
|[Kerry_blue_terrier,17,16,300,482]|
|[vizsla,112,93,276,236]           |
+----------------------------------+
only showing top 10 rows

None
+----------+
|size(dogs)|
+----------+
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
+----------+
only showing top 10 rows

+--------+---------------+-----+------+--------------------+----------+-----------------+
|  folder|       filename|width|height|                dogs|dog_pixels|      dog_percent|
+--------+---------------+-----+------+--------------------+---

+----------+
|size(dogs)|
+----------+
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
+----------+
only showing top 10 rows

